In [ ]:
import json
import os

import pandas as pd
import tqdm
from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)

In [ ]:
DETECTIONS_JSON = os.path.abspath("./resources/json/search_0_0015.json")
DATASET_CSV = os.path.abspath(
    "./data/interim/emptyNonEmptyDataset/emptyNonEmptyDatasetSplit.csv"
)
COVERAGE_FOLDER = os.path.abspath("./resources/megadetector_coverage")

In [ ]:
threshold = (
    os.path.basename(DETECTIONS_JSON).replace(
        os.path.basename(DETECTIONS_JSON).split("_")[0] + "_", ""
    )
).replace(".json", "")

name = "MegaDetector"
model = name + "_" + str(threshold)

with open(DETECTIONS_JSON, "r") as file:
    data = json.load(file)

In [ ]:
dataset = pd.read_csv(DATASET_CSV, sep=";")

In [ ]:
report = pd.DataFrame({})

for image in tqdm.tqdm(data["images"]):
    image_file = image["file"]
    indexes = dataset[dataset["file_name"] == image_file]

    if len(indexes) == 1:
        label = indexes["label"].iloc[0]
        binary_label = int(indexes["binary_label"].iloc[0])
        detector_label = 1 if image["max_detection_conf"] > 0.0 else 0
        subset = indexes["subset"].iloc[0]

        new_row = {
            "file_name": image_file,
            "label": label,
            "binary_label": binary_label,
            "detector_label": detector_label,
            "subset": subset,
        }
        report = pd.concat([report, pd.DataFrame([new_row])], ignore_index=True)

In [ ]:
os.makedirs(COVERAGE_FOLDER, exist_ok=True)
report.to_csv(
    os.path.join(COVERAGE_FOLDER, model + ".csv"),
    index=False,
    sep=";",
)